## Import stuff

In [2]:
# Jupyter shenanigans
%matplotlib widget
%config InlineBackend.figure_format='retina'

# Science 
import numpy as np
from scipy import interpolate, ndimage, signal

# Plotting and visualization
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Data tools
import h5py
import sqlite3

# MongoDB
from customImports import mongo 
client = mongo.client_connect()
db = mongo.database_connect(client,'Experiments')
collection = db.Thermography

# Misc
from datetime import datetime, timedelta
import importlib
import copy

# Set base directory for macOS
import os.path
dir = '/'.join(os.getcwd().split('/')[0:3])+'/'
print(dir)

/Users/howiechu/


## Import files

In [3]:
# Set file directory, experiment folder always contains A.mat for matlab array
# and exptname.sqlite or .csv (Ivium vs. kepco) for voltage data
thermdir = dir+'Desktop/Kokam/KKMF/SOH95/SOC60/KKMF_SOC60_4c50s_021220/'
ecf = 'KKMF_SOC60_4c50s_021220.sqlite'

In [4]:
# load IR video array
def reloadvid():
    A = h5py.File(thermdir+'A.mat','r')[('A')]
    # Transpose Array (some older sets are transposed for some reason)
    if A.shape[1]>A.shape[2]:
        A = np.transpose(A,(0,2,1))
    return A

A = reloadvid()

In [5]:
# load IVIUM sqlite database
conn = sqlite3.connect(thermdir+ecf)
cur = conn.cursor()
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")

# print table names if you need it
# for name in res:
#     print(name[0])

In [6]:
# Set thermometer temperature to calibrate ambient thermocouple
tref = 23.5
cur.execute('SELECT e FROM analog')
tcouple = cur.fetchall()
tcouple = np.asarray(tcouple[1::2])
tcouple = tcouple*200
tcouple = tcouple-abs(tcouple[0]-tref)

# Assemble echem data [time,amb. temp, current, voltage]
cur.execute('SELECT t,y,z FROM point')
EC = cur.fetchall()
EC = np.asarray(EC)
EC = np.hstack((EC,tcouple))
# restack columns to match when we used the KEPCO
EC = EC[:,[0,3,1,2]]

In [7]:
from customImports import thermography_functions as tf
# Reload module if changes are made
# importlib.reload(tf)
kokam = tf.thermography(A, 'Kokam')

## Plots DAQ data, change col to get other column data

In [8]:
kokam.plot_DAQ(EC, ylabel = 'Ambient temperature ($^{\circ}$C)', col = [1])
# kokam.plot_DAQ(EC, ylabel = 'Potential (V)', col = 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Importing and syncing timestamps

In [9]:
# IR camera timestamps
f = np.loadtxt(thermdir+'timestamps.csv', delimiter=',', dtype=str)
# Converts time string to datetime object
At = [datetime.strptime(time[4:],'%H:%M:%S.%f') for time in f]
# Grabs starting IR video frame's time of capture
irStart = datetime.timestamp(At[0])
# Converts all datetime objects to timestamps and places them into a list
At = np.asarray([datetime.timestamp(n) for n in At])

- Until the ivium can tell time correctly the measurement start time is useless but how to access it is here just in case.
- 2021/04/21 - Update : newest ivium software fixed time issues, not sure if it has milliseconds so use whatever you want, all of my data will have to use file creation time

```
# Pulls the time the Octostat starts the test 
cur.execute('SELECT start_time FROM measurement')
dstE = cur.fetchall()[0][0][11:]
dstE = datetime.timestamp(datetime.strptime(dstE,'%H:%M:%S'))
```

In [10]:
# Pulls the time the sqlite file was created
cur.execute('SELECT * FROM metadata')
iviumStart = cur.fetchall()[3][1]
iviumStart = datetime.timestamp(datetime.strptime(iviumStart, '%H-%M-%S-%f'))

# Compares start times of Ivium and the IR camera
checkStart = iviumStart-irStart

# Normalizes the thermography data and voltage data timestamps
# 0 is when the ivium starts collecting data
At = At-At[0]-checkStart

# Find when current starts to flow
# Finds where absolute value of current is greater than 0.2 the first time
off = np.where(np.absolute(EC[:,2])>0.2)[0][0]
timeOffset = EC[off,0]

# Time sync : 0 is now when current starts flowing for both devices
At = At-timeOffset
EC[:,0] = EC[:,0]-timeOffset

## Adjust ROI locations (full image)
Manually change parameters until rectangles align, then uncomment last line to save figure
- Sampling area : ndarray[row_index][column_index]
- With patches  : patches.thingyouwant((x,y),width,height,etc)

In [11]:
# Manually choose where the active battery material and ambient spot locations are
# Placed into list [top left corner row, top left corner col, number of rows, number of columns] or [y, x, height, width]
# Can probably do something like edge detection instead
Abatt = [135,117,52,114]
Aamb = [202,75,27,27]

## Plots the ROI that are chosen above

In [12]:
kokam.plot_ROI(Abatt, Aamb, frame = 2000)
# Save current figure, if wanted
# There is also a function to save all open figures near the end
# plt.savefig(thermdir+"ROI_return.pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Slice out ROI data for further processing

In [13]:
B, Amb = kokam.extract_ROI(Abatt, Aamb)
# Not necessary but might need to reload the original file if working backwards
B = copy.deepcopy(B)

### Camera calibration adjustment

In [14]:
B = kokam.ambient_calibration(B, Amb)

### Hot spot tracker

In [15]:
# Finds index of IR frame that references  t = 0
At0 = np.where(np.round(At) == 0)[0][0]
# Function to generate list of 2D max locations in (x,y) for n frames after t = 0
indMax = kokam.hotspot_location_array(B, At0, 2500)

In [16]:
# Average the x and y coordinate 
# (patchSize/2) is for making up for the dimension of the square
patchSize = 10
Ahot = np.round(np.mean(indMax[200:], axis=0)).astype(int)-int(patchSize/2)
# Acold is just defaulted to bottom left corner, shifted to match patchSize
Acold = [B[0].shape[0]-patchSize, 0]

print('Hot spot location (pixels): '+str(Ahot))
print('Cold spot location (pixels): '+str(Acold))

Hot spot location (pixels): [21 51]
Cold spot location (pixels): [42, 0]


### Plots location of hot spot at each time step using magma color map
- dark/blue = early
- orange/white = later

In [17]:
kokam.plot_extremes_tracker(B[0].shape, indMax, Ahot, Acold, patchSize)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plots the hot/cold spot rectangles over a sample frame from IR video

In [18]:
kokam.plot_battery_ROI(B, patchSize, 1000, Ahot, Acold)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# ROI data generation 
- takes surface average of the entire input array, and the slices provided

In [19]:
Thot, Tavg, Tcold = kokam.extract_data(B, Ahot, Acold, patchSize)

#### Convert $\Delta$T back to absolute using average ambient temperature 
#### (from ambient temperature thermocouple calibrated with thermometer)

In [20]:
# Add back the time average air temperature of the room (over time of experiment)
tAmbAvg = np.mean(EC[:,1])

Thot = Thot + tAmbAvg
Tavg = Tavg + tAmbAvg
Tcold = Tcold + tAmbAvg

# Combines the column vectors into an array
IR = np.stack((At,Thot,Tavg,Tcold), axis = 1)

# Interpolate all the things
- Camera freq is at 1 Hz
- Data logger is at 5 Hz (older experiments were at 1 Hz)
- linearly interpolate at 1 Hz

In [21]:
Time = np.arange(0,int(At[-1]),1)
Table = np.empty((Time.shape[0],IR.shape[1]+1))
Table[:,0] = Time
for x in range(1,4):
    Table[:,x] = np.interp(Time,IR[:,0],IR[:,x])
Table[:,-1] = np.interp(Time,EC[:,0],EC[:,3])

### Normalize starting temperatures
- shifts every column so that t=0 has the same temperatures for hot/cold/average

In [22]:
# Small adjustment to the starting temperatures
avgstart = np.mean(Table[0,1:4])
for x in range(1,4):
    Table[:,x] = Table[:,x]-Table[0,x]+avgstart
# np.savetxt(dir+expt+"thermographyData.csv", Table, delimiter=",", fmt='%1.3f')

# Plot

In [23]:
kokam.plot_processed_thermography(Table, endTime = 2500, adjustable = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Run these cells if there are bad calibrations in the data (jumps)

In [24]:
# Adjust which series to analyze and peak height to choose most appropriate fix
kokam.analyze_jumps(Table, 3, 0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fTable = kokam.execute_jumps(Table)

In [26]:
kokam.plot_processed_thermography(fTable, endTime = 2500, adjustable = True, figName = 'postFix')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
# np.savetxt(dir+expt+"thermographyData.csv", fTable, delimiter=",", fmt='%1.3f')

# Smoothing for video

In [28]:
# Associate At (timestamps) to the B matrix, 
# then interp1D to get interpolated temp data at relevant times
Ainterp = interpolate.interp1d(At, B, axis=0)

# Time is the same dimensions as COMSOL data length
evalTime = np.arange(0,int(At[-1]),1)

# Evaluate interpolation function
Binterp = Ainterp(evalTime)

# Convert back to absolute temperature using tAmbAvg from previous cells
Binterp = Binterp - Binterp[0] + tAmbAvg

# Smooth the matrix for visualization (gaussian blur)
Bsmooth = ndimage.filters.gaussian_filter(Binterp, 5)

In [29]:
# Use list of time steps you want to plot
plotTimes = [1000, 1500, 2100, 2400]
# Input number of columns you want the figure to have
kokam.plot_video_frames(Bsmooth, plotTimes, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Code to save all open figures into a single PDF

In [ ]:
def save_allfigs(filename, figs=None):
    '''
    Saves all open figures to a single pdf file if no figure labels are passed, 
    otherwise saves whatever strings are passed.
    
    Parameters:
        filename (str): filename for pdf
        figs (list of str): list of figure labels
        
    Returns:
        pdf file with a figure on each page
    '''
    with PdfPages('{}.pdf'.format(filename)) as pdf:
        if figs is None:
            figs = plt.get_figlabels()
        for fig in figs:
            tempFig = plt.figure(fig)
            pdf.savefig(tempFig, dpi = 300)

In [ ]:
save_allfigs('test123')

In [30]:

doc_update = {
    'directory':thermdir,
    'filename': {
        'IR': thermdir.split('/')[-2]+'test',
        'EC': ecf
    },
    'ROI': {
        'battery corner':Abatt[0:2],
        'battery height':Abatt[2],
        'battery width':Abatt[3],
        'ambient corner':Aamb[0:2],
        'ambient height':Aamb[2],
        'ambient width':Aamb[3]
    },
    'expt':{
        'cell':'Kokam',
        'cellID':thermdir.split('/')[-2].split('_')[0],
        'type':'squarewave',
        'SOH':95,
        'SOC':60,
        'c-rate':2,
        'period':100,
        'tref':tref,
        'PSU':'octostat'
    },
    'batteryROI': {
        'hot corner':Ahot.tolist(),
        'hot height':patchSize,
        'hot width':patchSize,
        'cold corner':Acold,
        'cold height':patchSize,
        'cold width':patchSize
    },
    'output':mongo.convert_ndarray_binary(fTable),
    'imageLocations': {
        'thermogramFull':thermdir+"ROI_return.pdf",
        'hotSpotMap':thermdir+"hotSpotTracked.pdf",
        'thermogramBatt':thermdir+"battery_ROI.pdf",
        'thermogramTime':thermdir+'thermogramTime.pdf',
        'outputPlot':thermdir+'dataPlot.pdf'
    },
    'notes': 'nothing of note',
    'dstCheck':dstCheck if ecf.split('.')[-1] in ['csv'] else None,
    'timeOffset':timeOffset
    
}

collection.update_one({'filename.IR':thermdir.split('/')[-2]+'test'}, {'$set':doc_update}, upsert=True)